# Assistants API Overview with AgentOps

This notebook has been adapted from this OpenAI Cookbook [example](https://cookbook.openai.com/examples/assistants_api_overview_python).

The new [Assistants API](https://platform.openai.com/docs/assistants/overview) is a stateful evolution of our [Chat Completions API](https://platform.openai.com/docs/guides/text-generation/chat-completions-api) meant to simplify the creation of assistant-like experiences, and enable developer access to powerful tools like Code Interpreter and Retrieval.

![Assistants API Diagram](https://github.com/AgentOps-AI/agentops/blob/main/examples/openai_examples/images/assistants_overview_diagram.png)

## Chat Completions API vs Assistants API

The primitives of the **Chat Completions API** are `Messages`, on which you perform a `Completion` with a `Model` (`gpt-3.5-turbo`, `gpt-4`, etc). It is lightweight and powerful, but inherently stateless, which means you have to manage conversation state, tool definitions, retrieval documents, and code execution manually.

The primitives of the **Assistants API** are

- `Assistants`, which encapsulate a base model, instructions, tools, and (context) documents,
- `Threads`, which represent the state of a conversation, and
- `Runs`, which power the execution of an `Assistant` on a `Thread`, including textual responses and multi-step tool use.

We'll take a look at how these can be used to create powerful, stateful experiences.


## Setup

> **Note**
> The Assistants API is currently in beta so the latest [Python SDK](https://github.com/openai/openai-python) is needed (`1.58.1` at time of writing) for this example.


In [ ]:
%pip install -U openai
%pip install -U agentops
%pip install -U python-dotenv

### Pretty Printing Helper


In [62]:
import json


def show_json(obj):
    display(json.loads(obj.model_dump_json()))

## Complete Example with Assistants API


### Assistants


The easiest way to get started with the Assistants API is through the [Assistants Playground](https://platform.openai.com/playground).


![Assistants Playground](https://github.com/AgentOps-AI/agentops/blob/main/examples/openai_examples/images/assistants_overview_assistants_playground.png)


Let's begin by creating an assistant! We'll create a Math Tutor just like in our [docs](https://platform.openai.com/docs/assistants/overview).


![Creating New Assistant](https://github.com/AgentOps-AI/agentops/blob/main/examples/openai_examples/images/assistants_overview_new_assistant.png)


You can view Assistants you've created in the [Assistants Dashboard](https://platform.openai.com/assistants).


![Assistants Dashboard](https://github.com/AgentOps-AI/agentops/blob/main/examples/openai_examples/images/assistants_overview_assistants_dashboard.png)


You can also create Assistants directly through the Assistants API. But we need to have the AgentOps and OpenAI API keys first.

You can get your OpenAI API key from the [OpenAI Dashboard](https://platform.openai.com/api-keys).

To obtain the AgentOps API key, signup for an account on [AgentOps](https://agentops.ai/) and create a project. After creating the project, you can now create an API key in the [Project Settings](https://app.agentops.ai/settings/projects).

Next, we'll set our API keys. There are several ways to do this, the code below is just the most foolproof way for the purposes of this notebook. It accounts for both users who use environment variables and those who just want to set the API Key here in this notebook.

1. Create an environment variable in a .env file or other method. By default, the AgentOps `init()` function will look for an environment variable named `AGENTOPS_API_KEY`. Or...

2. Replace `<your_agentops_key>` below and pass in the optional `api_key` parameter to the AgentOps `init(api_key=...)` function. Remember not to commit your API key to a public repo!

Now we are all set! Let's import the necessary libraries and initialize the AgentOps and OpenAI clients.

In [63]:
from openai import OpenAI
import agentops
from dotenv import load_dotenv
import os

load_dotenv()
AGENTOPS_API_KEY = os.getenv("AGENTOPS_API_KEY") or "<your_agentops_key>"
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") or "<your_openai_key>"

In [64]:
agentops.init(api_key=AGENTOPS_API_KEY, default_tags=["openai", "beta-assistants"],auto_start_session=True)
client = OpenAI(api_key=OPENAI_API_KEY)

(DEBUG) 🖇 AgentOps: Instrumentors have already been populated.
(DEBUG) 🖇 AgentOps: [DEBUG] BEFORE _make_span session.session - Current context: {'span_id': '79636eb80922dac8', 'trace_id': '15191e1a9ca916a408f7537d83c210c0', 'name': 'session.session', 'is_recording': <bound method Span.is_recording of _Span(name="session.session", context=SpanContext(trace_id=0x15191e1a9ca916a408f7537d83c210c0, span_id=0x79636eb80922dac8, trace_flags=0x01, trace_state=[], is_remote=False))>}
(DEBUG) 🖇 AgentOps: Started span: session.session (kind: session)
🖇 AgentOps: Session started: https://app.agentops.ai/drilldown?session_id=15191e1a-9ca9-16a4-08f7-537d83c210c0
(DEBUG) 🖇 AgentOps: [DEBUG] CREATED _make_span session.session - span_id: 4c582481db9c04be, parent: 79636eb80922dac8
🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=15191e1a-9ca9-16a4-08f7-537d83c210c0


Next, we'll create an Assistant which will be our Math Tutor.

In [65]:
assistant = client.beta.assistants.create(
    name="Math Tutor",
    instructions="You are a personal math tutor. Answer questions briefly, in a sentence or less.",
    model="gpt-4o-mini",
)
show_json(assistant)

{'id': 'asst_ElpoBZrn06hMdUp1LO5p2CFD',
 'created_at': 1742586617,
 'description': None,
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'metadata': {},
 'model': 'gpt-4o-mini',
 'name': 'Math Tutor',
 'object': 'assistant',
 'tools': [],
 'response_format': 'auto',
 'temperature': 1.0,
 'tool_resources': {'code_interpreter': None, 'file_search': None},
 'top_p': 1.0,
 'reasoning_effort': None}

Regardless of whether you create your Assistant through the Dashboard or with the API, you'll want to keep track of the Assistant ID. This is how you'll refer to your Assistant throughout Threads and Runs.


Next, we'll create a new Thread and add a Message to it. This will hold the state of our conversation, so we don't have re-send the entire message history each time.


### Threads


Create a new thread:


In [66]:
thread = client.beta.threads.create()
show_json(thread)

{'id': 'thread_uw17QRPJhQkAMtOTCGwKrBMp',
 'created_at': 1742586618,
 'metadata': {},
 'object': 'thread',
 'tool_resources': {'code_interpreter': None, 'file_search': None}}

Then add the Message to the thread:


In [67]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I need to solve the equation `3x + 11 = 14`. Can you help me?",
)
show_json(message)

{'id': 'msg_H3Hcs3msbCoNxPUsb9xX8WnP',
 'assistant_id': None,
 'attachments': [],
 'completed_at': None,
 'content': [{'text': {'annotations': [],
    'value': 'I need to solve the equation `3x + 11 = 14`. Can you help me?'},
   'type': 'text'}],
 'created_at': 1742586618,
 'incomplete_at': None,
 'incomplete_details': None,
 'metadata': {},
 'object': 'thread.message',
 'role': 'user',
 'run_id': None,
 'status': None,
 'thread_id': 'thread_uw17QRPJhQkAMtOTCGwKrBMp'}

> **Note**
> Even though you're no longer sending the entire history each time, you will still be charged for the tokens of the entire conversation history with each Run.


### Runs

Notice how the Thread we created is **not** associated with the Assistant we created earlier! Threads exist independently from Assistants, which may be different from what you'd expect if you've used ChatGPT (where a thread is tied to a model/GPT).

To get a completion from an Assistant for a given Thread, we must create a Run. Creating a Run will indicate to an Assistant it should look at the messages in the Thread and take action: either by adding a single response, or using tools.

> **Note**
> Runs are a key difference between the Assistants API and Chat Completions API. While in Chat Completions the model will only ever respond with a single message, in the Assistants API a Run may result in an Assistant using one or multiple tools, and potentially adding multiple messages to the Thread.

To get our Assistant to respond to the user, let's create the Run. As mentioned earlier, you must specify _both_ the Assistant and the Thread.


In [68]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)
show_json(run)

{'id': 'run_4iI0RxJyKGWYflcgcwQEqTbX',
 'assistant_id': 'asst_ElpoBZrn06hMdUp1LO5p2CFD',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1742586619,
 'expires_at': 1742587219,
 'failed_at': None,
 'incomplete_details': None,
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'last_error': None,
 'max_completion_tokens': None,
 'max_prompt_tokens': None,
 'metadata': {},
 'model': 'gpt-4o-mini',
 'object': 'thread.run',
 'parallel_tool_calls': True,
 'required_action': None,
 'response_format': 'auto',
 'started_at': None,
 'status': 'queued',
 'thread_id': 'thread_uw17QRPJhQkAMtOTCGwKrBMp',
 'tool_choice': 'auto',
 'tools': [],
 'truncation_strategy': {'type': 'auto', 'last_messages': None},
 'usage': None,
 'temperature': 1.0,
 'top_p': 1.0,
 'tool_resources': {},
 'reasoning_effort': None}

Unlike creating a completion in the Chat Completions API, **creating a Run is an asynchronous operation**. It will return immediately with the Run's metadata, which includes a `status` that will initially be set to `queued`. The `status` will be updated as the Assistant performs operations (like using tools and adding messages).

To know when the Assistant has completed processing, we can poll the Run in a loop. (Support for streaming is coming soon!) While here we are only checking for a `queued` or `in_progress` status, in practice a Run may undergo a [variety of status changes](https://platform.openai.com/docs/api-reference/runs/object#runs/object-status) which you can choose to surface to the user. (These are called Steps, and will be covered later.)


In [69]:
import time


def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run

In [70]:
run = wait_on_run(run, thread)
show_json(run)

{'id': 'run_4iI0RxJyKGWYflcgcwQEqTbX',
 'assistant_id': 'asst_ElpoBZrn06hMdUp1LO5p2CFD',
 'cancelled_at': None,
 'completed_at': 1742586621,
 'created_at': 1742586619,
 'expires_at': None,
 'failed_at': None,
 'incomplete_details': None,
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'last_error': None,
 'max_completion_tokens': None,
 'max_prompt_tokens': None,
 'metadata': {},
 'model': 'gpt-4o-mini',
 'object': 'thread.run',
 'parallel_tool_calls': True,
 'required_action': None,
 'response_format': 'auto',
 'started_at': 1742586620,
 'status': 'completed',
 'thread_id': 'thread_uw17QRPJhQkAMtOTCGwKrBMp',
 'tool_choice': 'auto',
 'tools': [],
 'truncation_strategy': {'type': 'auto', 'last_messages': None},
 'usage': {'completion_tokens': 36,
  'prompt_tokens': 66,
  'total_tokens': 102,
  'prompt_token_details': {'cached_tokens': 0},
  'completion_tokens_details': {'reasoning_tokens': 0}},
 'temperature': 1.0,
 'top_p': 1.0,
 'to

### Messages


Now that the Run has completed, we can list the Messages in the Thread to see what got added by the Assistant.


In [71]:
messages = client.beta.threads.messages.list(thread_id=thread.id)
show_json(messages)

(DEBUG) 🖇 AgentOps: Started span: openai.assistant.run (kind: unknown)
(DEBUG) 🖇 AgentOps: Ended span: openai.assistant.run (kind: unknown)
(DEBUG) 🖇 AgentOps: Ended span: openai.assistant.run (kind: unknown)


{'data': [{'id': 'msg_WAMEjtq4CGdR28jNx8wa8Vhy',
   'assistant_id': 'asst_ElpoBZrn06hMdUp1LO5p2CFD',
   'attachments': [],
   'completed_at': None,
   'content': [{'text': {'annotations': [],
      'value': 'Sure! Subtract 11 from both sides to get \\( 3x = 3 \\), then divide by 3 to find \\( x = 1 \\).'},
     'type': 'text'}],
   'created_at': 1742586620,
   'incomplete_at': None,
   'incomplete_details': None,
   'metadata': {},
   'object': 'thread.message',
   'role': 'assistant',
   'run_id': 'run_4iI0RxJyKGWYflcgcwQEqTbX',
   'status': None,
   'thread_id': 'thread_uw17QRPJhQkAMtOTCGwKrBMp'},
  {'id': 'msg_H3Hcs3msbCoNxPUsb9xX8WnP',
   'assistant_id': None,
   'attachments': [],
   'completed_at': None,
   'content': [{'text': {'annotations': [],
      'value': 'I need to solve the equation `3x + 11 = 14`. Can you help me?'},
     'type': 'text'}],
   'created_at': 1742586618,
   'incomplete_at': None,
   'incomplete_details': None,
   'metadata': {},
   'object': 'thread.messag

As you can see, Messages are ordered in reverse-chronological order – this was done so the most recent results are always on the first `page` (since results can be paginated). Do keep a look out for this, since this is the opposite order to messages in the Chat Completions API.


Let's ask our Assistant to explain the result a bit further!


In [72]:
# Create a message to append to our thread
message = client.beta.threads.messages.create(thread_id=thread.id, role="user", content="Could you explain this to me?")

# Execute our run
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)

# Wait for completion
wait_on_run(run, thread)

# Retrieve all the messages added after our last user message
messages = client.beta.threads.messages.list(thread_id=thread.id, order="asc", after=message.id)
show_json(messages)

(DEBUG) 🖇 AgentOps: Started span: openai.assistant.run (kind: unknown)
(DEBUG) 🖇 AgentOps: Ended span: openai.assistant.run (kind: unknown)
(DEBUG) 🖇 AgentOps: Ended span: openai.assistant.run (kind: unknown)


{'data': [{'id': 'msg_qS8ER2CuEovcezcm4ksQ4vOS',
   'assistant_id': 'asst_ElpoBZrn06hMdUp1LO5p2CFD',
   'attachments': [],
   'completed_at': None,
   'content': [{'text': {'annotations': [],
      'value': 'Of course! To solve the equation \\( 3x + 11 = 14 \\), follow these steps:\n\n1. **Isolate the term with \\( x \\)**: Subtract 11 from both sides to eliminate 11 on the left. This gives you:\n   \\[\n   3x + 11 - 11 = 14 - 11\n   \\]\n   Simplifying this, we get:\n   \\[\n   3x = 3\n   \\]\n\n2. **Solve for \\( x \\)**: Now, divide both sides by 3 to isolate \\( x \\):\n   \\[\n   \\frac{3x}{3} = \\frac{3}{3}\n   \\]\n   This simplifies to:\n   \\[\n   x = 1\n   \\]\n\nSo, the solution is \\( x = 1 \\).'},
     'type': 'text'}],
   'created_at': 1742586625,
   'incomplete_at': None,
   'incomplete_details': None,
   'metadata': {},
   'object': 'thread.message',
   'role': 'assistant',
   'run_id': 'run_5hso6XQCmns3Q5dSdcSM9QDS',
   'status': None,
   'thread_id': 'thread_uw17QRPJh

This may feel like a lot of steps to get a response back, especially for this simple example. However, you'll soon see how we can add very powerful functionality to our Assistant without changing much code at all!


### Example


Let's take a look at how we could potentially put all of this together. Below is all the code you need to use an Assistant you've created.

Since we've already created our Math Assistant, I've saved its ID in `MATH_ASSISTANT_ID`. I then defined two functions:

- `submit_message`: create a Message on a Thread, then start (and return) a new Run
- `get_response`: returns the list of Messages in a Thread


In [73]:
from openai import OpenAI

MATH_ASSISTANT_ID = assistant.id  # or a hard-coded ID like "asst-..."

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", "<your OpenAI API key if not set as env var>"))


def submit_message(assistant_id, thread, user_message):
    client.beta.threads.messages.create(thread_id=thread.id, role="user", content=user_message)
    return client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant_id,
    )


def get_response(thread):
    return client.beta.threads.messages.list(thread_id=thread.id, order="asc")

I've also defined a `create_thread_and_run` function that I can re-use (which is actually almost identical to the [`client.beta.threads.create_and_run`](https://platform.openai.com/docs/api-reference/runs/createThreadAndRun) compound function in our API ;) ). Finally, we can submit our mock user requests each to a new Thread.

Notice how all of these API calls are asynchronous operations; this means we actually get async behavior in our code without the use of async libraries! (e.g. `asyncio`)


In [74]:
def create_thread_and_run(user_input):
    thread = client.beta.threads.create()
    run = submit_message(MATH_ASSISTANT_ID, thread, user_input)
    return thread, run


# Emulating concurrent user requests
thread1, run1 = create_thread_and_run("I need to solve the equation `3x + 11 = 14`. Can you help me?")
thread2, run2 = create_thread_and_run("Could you explain linear algebra to me?")
thread3, run3 = create_thread_and_run("I don't like math. What can I do?")

# Now all Runs are executing...

Once all Runs are going, we can wait on each and get the responses.


In [75]:
# Pretty printing helper
def pretty_print(messages):
    print("# Messages")
    for m in messages:
        print(f"{m.role}: {m.content[0].text.value}")
    print()


# Waiting in a loop
def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run


# Wait for Run 1
run1 = wait_on_run(run1, thread1)
pretty_print(get_response(thread1))

# Wait for Run 2
run2 = wait_on_run(run2, thread2)
pretty_print(get_response(thread2))

# Wait for Run 3
run3 = wait_on_run(run3, thread3)
pretty_print(get_response(thread3))

# Thank our assistant on Thread 3 :)
run4 = submit_message(MATH_ASSISTANT_ID, thread3, "Thank you!")
run4 = wait_on_run(run4, thread3)
pretty_print(get_response(thread3))

(DEBUG) 🖇 AgentOps: Started span: openai.assistant.run (kind: unknown)
(DEBUG) 🖇 AgentOps: Ended span: openai.assistant.run (kind: unknown)
(DEBUG) 🖇 AgentOps: Ended span: openai.assistant.run (kind: unknown)


# Messages
user: I need to solve the equation `3x + 11 = 14`. Can you help me?
assistant: Sure! To solve for \( x \), subtract 11 from both sides: \( 3x = 3 \). Then, divide by 3: \( x = 1 \).



(DEBUG) 🖇 AgentOps: Started span: openai.assistant.run (kind: unknown)
(DEBUG) 🖇 AgentOps: Ended span: openai.assistant.run (kind: unknown)
(DEBUG) 🖇 AgentOps: Ended span: openai.assistant.run (kind: unknown)


# Messages
user: Could you explain linear algebra to me?
assistant: Linear algebra is the branch of mathematics that deals with vectors, vector spaces, linear transformations, and systems of linear equations.



(DEBUG) 🖇 AgentOps: Started span: openai.assistant.run (kind: unknown)
(DEBUG) 🖇 AgentOps: Ended span: openai.assistant.run (kind: unknown)
(DEBUG) 🖇 AgentOps: Ended span: openai.assistant.run (kind: unknown)


# Messages
user: I don't like math. What can I do?
assistant: Try to relate math to your interests or find practical applications to make it more engaging.



(DEBUG) 🖇 AgentOps: Started span: openai.assistant.run (kind: unknown)
(DEBUG) 🖇 AgentOps: Ended span: openai.assistant.run (kind: unknown)
(DEBUG) 🖇 AgentOps: Ended span: openai.assistant.run (kind: unknown)


# Messages
user: I don't like math. What can I do?
assistant: Try to relate math to your interests or find practical applications to make it more engaging.
user: Thank you!
assistant: You're welcome! If you have any more questions, feel free to ask!



Et voilà!

You may have noticed that this code is not actually specific to our math Assistant at all... this code will work for any new Assistant you create simply by changing the Assistant ID! That is the power of the Assistants API.


## Tools

A key feature of the Assistants API is the ability to equip our Assistants with Tools, like Code Interpreter, Retrieval, and custom Functions. Let's take a look at each.

### Code Interpreter

Let's equip our Math Tutor with the [Code Interpreter](https://platform.openai.com/docs/assistants/tools/code-interpreter) tool, which we can do from the Dashboard...


![Enabling code interpreter](https://github.com/AgentOps-AI/agentops/blob/main/examples/openai_examples/images/assistants_overview_enable_code_interpreter.png)


...or the API, using the Assistant ID.


In [76]:
assistant = client.beta.assistants.update(
    MATH_ASSISTANT_ID,
    tools=[{"type": "code_interpreter"}],
)
show_json(assistant)

{'id': 'asst_ElpoBZrn06hMdUp1LO5p2CFD',
 'created_at': 1742586617,
 'description': None,
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'metadata': {},
 'model': 'gpt-4o-mini',
 'name': 'Math Tutor',
 'object': 'assistant',
 'tools': [{'type': 'code_interpreter'}],
 'response_format': 'auto',
 'temperature': 1.0,
 'tool_resources': {'code_interpreter': {'file_ids': []}, 'file_search': None},
 'top_p': 1.0,
 'reasoning_effort': None}

Now, let's ask the Assistant to use its new tool.


In [77]:
thread, run = create_thread_and_run("Generate the first 20 fibbonaci numbers with code.")
run = wait_on_run(run, thread)
pretty_print(get_response(thread))

(DEBUG) 🖇 AgentOps: Started span: openai.assistant.run (kind: unknown)
(DEBUG) 🖇 AgentOps: Ended span: openai.assistant.run (kind: unknown)
(DEBUG) 🖇 AgentOps: Ended span: openai.assistant.run (kind: unknown)


# Messages
user: Generate the first 20 fibbonaci numbers with code.
assistant: The first 20 Fibonacci numbers are:  
\[ 0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181 \]



And that's it! The Assistant used Code Interpreter in the background, and gave us a final response.

For some use cases this may be enough – however, if we want more details on what precisely an Assistant is doing we can take a look at a Run's Steps.

### Steps


A Run is composed of one or more Steps. Like a Run, each Step has a `status` that you can query. This is useful for surfacing the progress of a Step to a user (e.g. a spinner while the Assistant is writing code or performing retrieval).


In [78]:
run_steps = client.beta.threads.runs.steps.list(thread_id=thread.id, run_id=run.id, order="asc")

Let's take a look at each Step's `step_details`.


In [79]:
for step in run_steps.data:
    step_details = step.step_details
    print(json.dumps(show_json(step_details), indent=4))

{'tool_calls': [{'id': 'call_0tCXMy7C6207VwnBgSBw95tb',
   'code_interpreter': {'input': 'def fibonacci(n):\r\n    fib_sequence = [0, 1]\r\n    for i in range(2, n):\r\n        next_fib = fib_sequence[-1] + fib_sequence[-2]\r\n        fib_sequence.append(next_fib)\r\n    return fib_sequence\r\n\r\n# Generate the first 20 Fibonacci numbers\r\nfirst_20_fib = fibonacci(20)\r\nfirst_20_fib',
    'outputs': []},
   'type': 'code_interpreter'}],
 'type': 'tool_calls'}

null


{'message_creation': {'message_id': 'msg_HHSmc6qRqBNy61nSCRWcwIvX'},
 'type': 'message_creation'}

null


We can see the `step_details` for two Steps:

1. `tool_calls` (plural, since it could be more than one in a single Step)
2. `message_creation`

The first Step is a `tool_calls`, specifically using the `code_interpreter` which contains:

- `input`, which was the Python code generated before the tool was called, and
- `output`, which was the result of running the Code Interpreter.

The second Step is a `message_creation`, which contains the `message` that was added to the Thread to communicate the results to the user.


### Retrieval

Another powerful tool in the Assistants API is [Retrieval](https://platform.openai.com/docs/assistants/tools/knowledge-retrieval): the ability to upload files that the Assistant will use as a knowledge base when answering questions. This can also be enabled from the Dashboard or the API, where we can upload files we want to be used.


![Enabling retrieval](https://github.com/AgentOps-AI/agentops/blob/main/examples/openai_examples/images/assistants_overview_enable_retrieval.png)


In [80]:
# Upload the file
file = client.files.create(
    file=open(
        "language_models_are_unsupervised_multitask_learners.pdf",
        "rb",
    ),
    purpose="assistants",
)
# Update Assistant
assistant = client.beta.assistants.update(
    MATH_ASSISTANT_ID,
    tools=[{"type": "code_interpreter"}],
    tool_resources={"code_interpreter": {"file_ids": [file.id]}},
)
show_json(assistant)

{'id': 'asst_ElpoBZrn06hMdUp1LO5p2CFD',
 'created_at': 1742586617,
 'description': None,
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'metadata': {},
 'model': 'gpt-4o-mini',
 'name': 'Math Tutor',
 'object': 'assistant',
 'tools': [{'type': 'code_interpreter'}],
 'response_format': 'auto',
 'temperature': 1.0,
 'tool_resources': {'code_interpreter': {'file_ids': ['file-6jj1Aik1ZvocsraUDL4zZD']},
  'file_search': None},
 'top_p': 1.0,
 'reasoning_effort': None}

In [ ]:
thread, run = create_thread_and_run(
    "What are some cool math concepts behind this ML paper pdf? Explain in two sentences."
)
run = wait_on_run(run, thread)
pretty_print(get_response(thread))

> **Note**
> There are more intricacies in Retrieval, like [Annotations](https://platform.openai.com/docs/assistants/how-it-works/managing-threads-and-messages), which may be covered in another cookbook.


### Functions

As a final powerful tool for your Assistant, you can specify custom [Functions](https://platform.openai.com/docs/assistants/tools/function-calling) (much like the [Function Calling](https://platform.openai.com/docs/guides/function-calling) in the Chat Completions API). During a Run, the Assistant can then indicate it wants to call one or more functions you specified. You are then responsible for calling the Function, and providing the output back to the Assistant.

Let's take a look at an example by defining a `display_quiz()` Function for our Math Tutor.

This function will take a `title` and an array of `question`s, display the quiz, and get input from the user for each:

- `title`
- `questions`
  - `question_text`
  - `question_type`: [`MULTIPLE_CHOICE`, `FREE_RESPONSE`]
  - `choices`: ["choice 1", "choice 2", ...]

Unfortunately I don't know how to get user input within a Python Notebook, so I'll be mocking out responses with `get_mock_response...`. This is where you'd get the user's actual input.


In [51]:
def get_mock_response_from_user_multiple_choice():
    return "a"


def get_mock_response_from_user_free_response():
    return "I don't know."


def display_quiz(title, questions):
    print("Quiz:", title)
    print()
    responses = []

    for q in questions:
        print(q["question_text"])
        response = ""

        # If multiple choice, print options
        if q["question_type"] == "MULTIPLE_CHOICE":
            for i, choice in enumerate(q["choices"]):
                print(f"{i}. {choice}")
            response = get_mock_response_from_user_multiple_choice()

        # Otherwise, just get response
        elif q["question_type"] == "FREE_RESPONSE":
            response = get_mock_response_from_user_free_response()

        responses.append(response)
        print()

    return responses

Here's what a sample quiz would look like:


In [52]:
responses = display_quiz(
    "Sample Quiz",
    [
        {"question_text": "What is your name?", "question_type": "FREE_RESPONSE"},
        {
            "question_text": "What is your favorite color?",
            "question_type": "MULTIPLE_CHOICE",
            "choices": ["Red", "Blue", "Green", "Yellow"],
        },
    ],
)
print("Responses:", responses)

Quiz: Sample Quiz

What is your name?

What is your favorite color?
0. Red
1. Blue
2. Green
3. Yellow

Responses: ["I don't know.", 'a']


Now, let's define the interface of this function in JSON format, so our Assistant can call it:


In [53]:
function_json = {
    "name": "display_quiz",
    "description": "Displays a quiz to the student, and returns the student's response. A single quiz can have multiple questions.",
    "parameters": {
        "type": "object",
        "properties": {
            "title": {"type": "string"},
            "questions": {
                "type": "array",
                "description": "An array of questions, each with a title and potentially options (if multiple choice).",
                "items": {
                    "type": "object",
                    "properties": {
                        "question_text": {"type": "string"},
                        "question_type": {
                            "type": "string",
                            "enum": ["MULTIPLE_CHOICE", "FREE_RESPONSE"],
                        },
                        "choices": {"type": "array", "items": {"type": "string"}},
                    },
                    "required": ["question_text"],
                },
            },
        },
        "required": ["title", "questions"],
    },
}

Once again, let's update our Assistant either through the Dashboard or the API.


![Enabling custom function](https://github.com/AgentOps-AI/agentops/blob/main/examples/openai_examples/images/assistants_overview_enable_function.png)

> **Note**
> Pasting the function JSON into the Dashboard was a bit finicky due to indentation, etc. I just asked ChatGPT to format my function the same as one of the examples on the Dashboard :).


In [54]:
assistant = client.beta.assistants.update(
    MATH_ASSISTANT_ID,
    tools=[
        {"type": "code_interpreter"},
        {"type": "function", "function": function_json},
    ],
)
show_json(assistant)

{'id': 'asst_RPYHIeGQKlcD6nsIVfGGABG9',
 'created_at': 1742586485,
 'description': None,
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'metadata': {},
 'model': 'gpt-4o-mini',
 'name': 'Math Tutor',
 'object': 'assistant',
 'tools': [{'type': 'code_interpreter'},
  {'function': {'name': 'display_quiz',
    'description': "Displays a quiz to the student, and returns the student's response. A single quiz can have multiple questions.",
    'parameters': {'type': 'object',
     'properties': {'title': {'type': 'string'},
      'questions': {'type': 'array',
       'description': 'An array of questions, each with a title and potentially options (if multiple choice).',
       'items': {'type': 'object',
        'properties': {'question_text': {'type': 'string'},
         'question_type': {'type': 'string',
          'enum': ['MULTIPLE_CHOICE', 'FREE_RESPONSE']},
         'choices': {'type': 'array', 'items': {'type': 'string'}}},
       

And now, we ask for a quiz.


In [55]:
thread, run = create_thread_and_run(
    "Make a quiz with 2 questions: One open ended, one multiple choice. Then, give me feedback for the responses."
)
run = wait_on_run(run, thread)
run.status

'requires_action'

Now, however, when we check the Run's `status` we see `requires_action`! Let's take a closer look.


In [56]:
show_json(run)

{'id': 'run_NTIvvHANWWpcdS1tWRE172mT',
 'assistant_id': 'asst_RPYHIeGQKlcD6nsIVfGGABG9',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1742586538,
 'expires_at': 1742587138,
 'failed_at': None,
 'incomplete_details': None,
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'last_error': None,
 'max_completion_tokens': None,
 'max_prompt_tokens': None,
 'metadata': {},
 'model': 'gpt-4o-mini',
 'object': 'thread.run',
 'parallel_tool_calls': True,
 'required_action': {'submit_tool_outputs': {'tool_calls': [{'id': 'call_LhxjmdyANSlkhxgCwrzz6KZJ',
     'function': {'arguments': '{"title": "Math Quiz", "questions": [{"question_text": "Explain what the Pythagorean theorem states and when it is used.", "question_type": "FREE_RESPONSE"}, {"question_text": "What is the value of 8^2 + 6^2? \\nA) 100\\nB) 64\\nC) 36\\nD) 80", "question_type": "MULTIPLE_CHOICE", "choices": ["A) 100", "B) 64", "C) 36", "D) 80"]}]}',
      'name': 'di

The `required_action` field indicates a Tool is waiting for us to run it and submit its output back to the Assistant. Specifically, the `display_quiz` function! Let's start by parsing the `name` and `arguments`.

> **Note**
> While in this case we know there is only one Tool call, in practice the Assistant may choose to call multiple tools.


In [57]:
# Extract single tool call
tool_call = run.required_action.submit_tool_outputs.tool_calls[0]
name = tool_call.function.name
arguments = json.loads(tool_call.function.arguments)

print("Function Name:", name)
print("Function Arguments:")
arguments

Function Name: display_quiz
Function Arguments:


{'title': 'Math Quiz',
 'questions': [{'question_text': 'Explain what the Pythagorean theorem states and when it is used.',
   'question_type': 'FREE_RESPONSE'},
  {'question_text': 'What is the value of 8^2 + 6^2? \nA) 100\nB) 64\nC) 36\nD) 80',
   'question_type': 'MULTIPLE_CHOICE',
   'choices': ['A) 100', 'B) 64', 'C) 36', 'D) 80']}]}

Now let's actually call our `display_quiz` function with the arguments provided by the Assistant:


In [58]:
responses = display_quiz(arguments["title"], arguments["questions"])
print("Responses:", responses)

Quiz: Math Quiz

Explain what the Pythagorean theorem states and when it is used.

What is the value of 8^2 + 6^2? 
A) 100
B) 64
C) 36
D) 80
0. A) 100
1. B) 64
2. C) 36
3. D) 80

Responses: ["I don't know.", 'a']


Great! (Remember these responses are the one's we mocked earlier. In reality, we'd be getting input from the back from this function call.)

Now that we have our responses, let's submit them back to the Assistant. We'll need the `tool_call` ID, found in the `tool_call` we parsed out earlier. We'll also need to encode our `list`of responses into a `str`.


In [59]:
tool_outputs = []
tool_calls = run.required_action.submit_tool_outputs.tool_calls

for tool_call in tool_calls:
    arguments = json.loads(tool_call.function.arguments)
    responses = display_quiz(arguments["title"], arguments["questions"])
    tool_outputs.append(
        {
            "tool_call_id": tool_call.id,
            "output": json.dumps(responses),
        }
    )

Quiz: Math Quiz

Explain what the Pythagorean theorem states and when it is used.

What is the value of 8^2 + 6^2? 
A) 100
B) 64
C) 36
D) 80
0. A) 100
1. B) 64
2. C) 36
3. D) 80

Quiz: Math Quiz

Explain what the Pythagorean theorem states and when it is used.

What is the value of 8^2 + 6^2? 
A) 100
B) 64
C) 36
D) 80
0. A) 100
1. B) 64
2. C) 36
3. D) 80



In [60]:
run = client.beta.threads.runs.submit_tool_outputs(thread_id=thread.id, run_id=run.id, tool_outputs=tool_outputs)
show_json(run)

{'id': 'run_NTIvvHANWWpcdS1tWRE172mT',
 'assistant_id': 'asst_RPYHIeGQKlcD6nsIVfGGABG9',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1742586538,
 'expires_at': 1742587138,
 'failed_at': None,
 'incomplete_details': None,
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'last_error': None,
 'max_completion_tokens': None,
 'max_prompt_tokens': None,
 'metadata': {},
 'model': 'gpt-4o-mini',
 'object': 'thread.run',
 'parallel_tool_calls': True,
 'required_action': None,
 'response_format': 'auto',
 'started_at': 1742586539,
 'status': 'queued',
 'thread_id': 'thread_NplU8C6YUpGBO87xgcBlzV7S',
 'tool_choice': 'auto',
 'tools': [{'type': 'code_interpreter'},
  {'function': {'name': 'display_quiz',
    'description': "Displays a quiz to the student, and returns the student's response. A single quiz can have multiple questions.",
    'parameters': {'type': 'object',
     'properties': {'title': {'type': 'string'},
      'qu

We can now wait for the Run to complete once again, and check our Thread!


In [61]:
run = wait_on_run(run, thread)
pretty_print(get_response(thread))

(DEBUG) 🖇 AgentOps: Started span: openai.assistant.run (kind: unknown)
(DEBUG) 🖇 AgentOps: Ended span: openai.assistant.run (kind: unknown)
(DEBUG) 🖇 AgentOps: Ended span: openai.assistant.run (kind: unknown)


# Messages
user: Make a quiz with 2 questions: One open ended, one multiple choice. Then, give me feedback for the responses.
assistant: Here are the quiz questions:

1. Explain what the Pythagorean theorem states and when it is used.
2. What is the value of \(8^2 + 6^2\)? 
   - A) 100
   - B) 64
   - C) 36
   - D) 80

### Feedback:
1. **Open-ended response:** It's okay to not remember; the Pythagorean theorem relates the lengths of the sides of a right triangle, stating that \(a^2 + b^2 = c^2\), where \(c\) is the hypotenuse.
2. **Multiple choice response:** The answer is A) 100, as \(8^2 + 6^2 = 64 + 36 = 100\). 

Feel free to ask if you have any questions or need more clarification!



Woohoo 🎉

## Conclusion

We covered the basics of the Assistants API using OpenAI's Python SDK and AgentOps for observability.

For more information, check out the Assistants API deep [deep dive](https://platform.openai.com/docs/assistants/deep-dive) guide and its [documentation](https://platform.openai.com/docs/api-reference/assistants).